# MovieLens dataset

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip /content/ml-1m.zip

--2022-04-13 07:58:19--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  10.7MB/s    in 0.5s    

2022-04-13 07:58:20 (10.7 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  /content/ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


## Dependencies and config 

In [ ]:
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [953 kB]
Hit:13 http://ppa.lau

In [ ]:
!pip install -q findspark
!pip install recommenders
!pip install surprise

     |████████████████████████████████| 335 kB 7.9 MB/s 
     |████████████████████████████████| 11.8 MB 26.1 MB/s 
     |████████████████████████████████| 4.0 MB 33.8 MB/s 
     |████████████████████████████████| 1.5 MB 46.0 MB/s 
     |████████████████████████████████| 12.4 MB 37.1 MB/s 
     |████████████████████████████████| 197 kB 69.0 MB/s 
     |████████████████████████████████| 61 kB 8.6 MB/s 
     |████████████████████████████████| 310 kB 59.8 MB/s 
     |████████████████████████████████| 636 kB 50.2 MB/s 
     |████████████████████████████████| 749 kB 50.7 MB/s 
     |████████████████████████████████| 10.9 MB 46.9 MB/s 
     |████████████████████████████████| 381 kB 49.3 MB/s 
     |████████████████████████████████| 6.5 MB 43.4 MB/s 
     |████████████████████████████████| 895 kB 36.2 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705346 sha256=ec394a697fd64cecd6377a21e60167f3d7

In [ ]:
import os
import numpy as np
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
#Use as many threads as your spark local machine have
#spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder.master("local[4]")\
          .appName("StructureAPI")\
          .config('spark.driver.memory','15g')\
          .config('spark.submit.deployMode','client')\
          .config('spark.executor.memory','2g')\
          .config('spark.executor.cores',4)\
          .config('spark.yarn.queue','short')\
          .getOrCreate() 

In [ ]:
!ls

ml-1m	   sample_data		      spark-3.2.1-bin-hadoop3.2.tgz
ml-1m.zip  spark-3.2.1-bin-hadoop3.2


## Apply ALS model and compute RSME for the predictions

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField
from recommenders.datasets.spark_splitters import spark_random_split

In [ ]:
lines = spark.read.text("/content/ml-1m/ratings.dat").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId = int(p[0]), movieId = int(p[1]),
                                     rating = float(p[2])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])
#(training, test) = spark_random_split(ratings, ratio = 0.8, seed = 42)


# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter = 10, regParam = 0.01, userCol = "userId", itemCol = "movieId", ratingCol = "rating",
          coldStartStrategy = "drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating",
                                predictionCol = "prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.show(truncate=False)

Root-mean-square error = 0.8913636647004352


/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                          |
+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |[{2426, 9.214789}, {1421, 8.092667}, {2834, 7.309105}, {2992, 6.9514837}, {1496, 6.684358}, {960, 6.658357}, {2674, 6.5592623}, {2055, 6.5560703}, {250, 6.510674}, {1695, 6.48783}]     |
|12    |[{1349, 9.841156}, {1471, 9.528682}, {3117, 9.182364}, {392, 9.087875}, {2579, 8.336916}, {3776, 8.303175}, {702, 8.27852}, {1420, 8.218352}, {3540, 8.212985}, {3636, 8.052603}]        |
|22    |[{2305, 5.8702965

## NDCG Metric

In [ ]:
from recommenders.evaluation.spark_evaluation import SparkRankingEvaluation

In [ ]:
rank_eval = SparkRankingEvaluation(test, predictions, k = 10, col_user="userId", col_item="movieId", 
                                    col_rating="rating", col_prediction="prediction",
                                    relevancy_method="top_k")

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
print("NDCG:", round(rank_eval.ndcg_at_k(), 12) )

NDCG: 0.999957168271


## Hit Rate Metric


In [ ]:
import pandas as pd
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise import Reader
from surprise import Dataset
from surprise import accuracy

from collections import defaultdict

In [ ]:
#convert spark DF to pandas DF
ratings_HitRate = ratings.toPandas()

# pandas DF to Dataset Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_HitRate[['userId', 'movieId', 'rating']], reader)

### ALS in Spark and applying LOOCV for data train

In [ ]:
HR_ALS_spark = ALS(maxIter=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
LOOCV = LeaveOneOut(n_splits=1, random_state=1)

for trainSet, testSet in LOOCV.split(data):
  #Train model
  newTrainSet = spark.createDataFrame(trainSet.build_testset()).selectExpr("_1 as userId", "_2 as movieId", "_3 as rating")
  model = HR_ALS_spark.fit(newTrainSet)
  #Predict
  newTestSet = spark.createDataFrame(testSet).selectExpr("_1 as userId", "_2 as movieId", "_3 as rating")
  leftOutPredictions = model.transform(newTestSet).collect()
  #Build prediction for all rating
  bigTestSet = trainSet.build_anti_testset(0)
  newBigTestSet = spark.createDataFrame(bigTestSet).selectExpr("_1 as userId", "_2 as movieId", "_3 as rating")
  allPredictions = model.transform(newBigTestSet)

In [ ]:
list_allPredictions = allPredictions.collect()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 38256)


### GET TOP N

In [ ]:
def GetTopN(predictions, n=10, minimumRating=4.0):
    topN = defaultdict(list)

    for userID, movieID, actualRating, estimatedRating in predictions:
        if (estimatedRating >= minimumRating):
            topN[int(userID)].append((int(movieID), estimatedRating))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    return topN

In [ ]:
#compute top 10 recom
topNPredicted = GetTopN(list_allPredictions, n=10)

### Calculate hit rate

In [ ]:
def hit_rate(topNPredicted, leftOutPred):
    hits = 0
    total = 0

 # For each left-out rating
    for leftOut in leftOutPred:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top 10 for this user?
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (int(leftOutMovieID) == int(movieID)):
                hit = True
                break
        if (hit) :
            hits += 1

        total += 1

    # Compute overall precision
    return hits/total

In [ ]:
print("Hit rate: ", round(hit_rate(topNPredicted, leftOutPredictions), 12))

Hit rate:  0.0
